# MXFP4 Training Example

## Check GPU Availability

In [1]:
!nvidia-smi

Sun Aug 24 05:54:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8              1W /   78W |      31MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = ",".join([f"{i+DEVICE_NUM}" for i in range(0, ADDITIONAL_GPU+1)])
environ["CUDA_VISIBLE_DEVICES"]

'0'

## Imports

In [3]:
from os import path

import torch
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

import wandb
from tqdm.auto import tqdm

In [4]:
if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda")  # torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

INFO: Using device - cuda


In [5]:
PROJECT_NAME = "MXFP4_Example"
RUN_NAME = "Qwen3_8B_From_Scratch"

# WandB Initialization
wandb.init(project=PROJECT_NAME, name=RUN_NAME)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: brew (brew-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Define Dataset

In [6]:
dataset_id = "Trelis/tiny-shakespeare"

In [7]:
from datasets import load_dataset

# 셰익스피어 데이터셋
dataset = load_dataset(dataset_id)

In [8]:
dataset['train'][0]

{'Text': "First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this i

In [9]:
dataset['train'][0].keys()

dict_keys(['Text'])

## Load Model

In [10]:
from transformers import Qwen3ForCausalLM, AutoTokenizer

In [11]:
reference_model_id = "Qwen/Qwen3-0.6B"

In [12]:
reference_tokenizer = AutoTokenizer.from_pretrained(reference_model_id, use_fast=True)
tokenized_datasets = dataset.map(lambda data: reference_tokenizer(data["Text"], truncation=True, padding=True), batched=True)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [15]:
import mx